In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numdifftools as nd

In [ ]:
def f(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def opt_step(x1, x2, eps):
    left = x1
    right = x2
    alpha = (1 + np.sqrt(5))/2
    x1 = right - (right - left)/alpha
    x2 = left + (right - left)/alpha
    f1 = f(x + x1*p)
    f2 = f(x + x2*p)
    while right - left > eps:
        if f1 > f2:
            left = x1
            x1 = x2
            x2 = left + (right - left)/alpha
            f1 = f2
            f2 = f(x + x2*p)
        else:
            right = x2
            x2 = x1
            x1 = right - (right - left)/alpha
            f2 = f1
            f1 = f(x + x1*p)
        step = (left + right)/2
    return step

x = np.array([-0.5, 0.5])

start = x

g = nd.Gradient(f)(x)
B = np.identity(x.shape[0])
p = -B@g
gamma = 0.01

res_sp = minimize(f, x)
sp_min = res_sp.fun

lim = 30

history = np.array([[x[0], x[1], f(x)]])

for _ in range(lim):
    x_old = x
    g_old = g
    x = x + gamma*p
    g = nd.Gradient(f)(x)
    s = g - g_old
    y = x - x_old
    B = B + np.outer(y - B@s,y - B@s)/np.inner(y - B@s,s)
    p = -B@g
    gamma = opt_step(0, 2, 0.00000001)
    history = np.append(history, [[x[0], x[1], f(x)]], axis = 0)

print('obtained f: ', f(x), 'scipy_solution: ', sp_min)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
X, Y = np.meshgrid(np.linspace(-1, 1.5, 100), np.linspace(-1, 1.5, 100))
Z = f([X, Y])
plt.contour(X, Y, Z, levels = 100, cmap = 'viridis', alpha = 0.8)
plt.plot(history[:, 0], history[:, 1], color = 'red', label = "Descent path")
plt.scatter(1, 1, color = 'blue', label = "Minimum [1, 1]", zorder = 5)
plt.scatter(-0.5, 0.5, color = 'black', label = 'Start ' + str(start), zorder = 5)

plt.xlabel("x[0]")
plt.ylabel("x[1]")
plt.legend()
plt.grid()
plt.show()